# Segmentace obrazu - text a jeho čtení
Cvičení je zaměřené na správné využití osvětlení při nasvícení objektu a následné využití metod pro segmentaci obrazu. Novinkou ve cvičení je využití OCR (optical character recognition) k převodu obrazu na text.

<img src="images/ocr.png" width="300">

Ke čtení je využita knihovna [**Tesseract OCR**](https://github.com/tesseract-ocr/tesseract).

### Import knihoven a konfigurace

In [ ]:
%run ../svz.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Seznam funkcí pro přehlednost:
- [`connect_camera(...)`](../svz.ipynb#connect_functions)
- [`show_images(...)`](../svz.ipynb#show_functions)


- [`to_gray(...)`](../svz.ipynb#preprocessing_functions)
- [`to_hsv(...)`](../svz.ipynb#preprocessing_functions)
- [`negative(...)`](../svz.ipynb#preprocessing_functions)
- [`crop(...)`](../svz.ipynb#preprocessing_functions)


- [`segmentation_one_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_auto_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_two_thresholds(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_adaptive_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`find_contours(...)`](../svz.ipynb#segmentation_functions)


- [`filtration_median(...)`](../svz.ipynb#filtration_functions)
- [`fill_holes(...)`](../svz.ipynb#filtration_functions)


- [`ocr(...)`](../svz.ipynb#ocr)


- [`to_intensity(...)`](../svz.ipynb#others)

---

### Úkol
Zvolte vhodné funkce pro segmentaci obrazu a přečtěte text na obrazu snímaném kamerou.

Pro testování algoritmu využijte funkci `algorithm()`. Funkce je vložena do snímání kamery. Je tak možné v reálném čase pozorovat, co se s obrazem děje. Po změně obsahu funkce je však potřeba načíst znovu i blok pro spuštění snímání z kamery.

Po vyladění algoritmu, využijte obrázek z kamery a nechte ho přečíst OCR. V případě, že výsledek bude po porovnání `True`, úkol jste splnili.

#### 1) Nastavte správně kameru.

In [ ]:
serial_number = '' ### vlastní sériové číslo kamery
width = 1280
height = 720

camera = ...(serial_number) ###

#### 2) Doplňte funkční verze algoritmů pro nasvícení jednotlivými typy světel.
Výstupem každého algoritmu by měl být binární obraz, kde pozadí má černou barvu a objekty (popředí) bílou. Obrázky uložte.

**2a)** Funkce pro segmentaci referenčního obrazce (černo-bílý). Osvětlení žádné navíc (při nízkých světelných podmínkách lze využít světlo DOME).

In [ ]:
def algorithm_reference(img):
    ###
    
    return ... ###

**2b)** Funkce pro segmentaci obrazce s využitím osvětlení pomocí dvou plošných světel typu bar.

In [ ]:
def algorithm_two_bars(img):
    ###
    
    return ... ###

**2c*)** Dobrovolná funkce pro segmentaci obrazce s využitím osvětlení typu dark field (konzole se 4 bar světly).

In [ ]:
def algorithm_console(img):
    ###
    
    return ... ###

Definuje obsah funkce pro zpracování obrazu, konkrétně jeho segmentaci. Je třeba měnit právě upravovaný algoritmus.

In [ ]:
def impro(img):
    res = algorithm_reference(img) ###

    show_camera_window(res, 1)
    return res

#### 3) Spusťte snímání a testujte algoritmus pro zpracování obrazu.

Info k oknu:
- Pro spuštění okna kamery je zapotřebí kliknout na tlačítko `Run Interact`. 
- Okno kamery se vypne stisknutím tlačítka `q`.
- Obrázek se uloží stisknutím tlačítka `s`.
- Po kliku levého tlačítka myši se vytisknou souřadnice místa v obraze.
- Pro projevení změn v nastavení pomocí GUI prvků je nutné ukončit okno kamery a znovu ho spustit.

In [ ]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
    
viewer = BaslerOpenCVViewer(camera)
viewer.set_features(FEATURES)
viewer.set_impro_function(impro)
viewer.run_interaction_continuous_shot(window_size=(width, height))

#### 4) Využijte funkční algoritmus na uložené obrázky z kamery.
**POZOR:** Dokud nebude vstupní obraz vypadat následovně, **NEPOUŠTĚJTE** se do další části.

<img src="images/fit_preprocessed.png" width="200">

In [ ]:
# Celkem 3 obrázky
image_paths = [
    ### 
]

# Načtení obrazů
images_preprocessed = []
for image_path in image_paths:
    images_preprocessed.append(to_gray(load_image((image_path)))

#### 5) Využijte OCR na výsledný binární obrázek

In [ ]:
texts = []
for image in images_preprocessed:
    text = ...(image) ###
    print(text)
    texts.append(text)

#### 6) Zkontrolujte správnost

In [ ]:
ref_text = 'FIT'

for i, text in enumerate(texts):
    print('Přečtený text je: ' + text)
    
    if text == ref_text:
        print('-> Úkol ' + str(i+1) + ' jste splnili!')
    else:
        print('-> Úkol je třeba dál ladit ...')    
    print('')